In [1]:
#uncomment below to install gradio library
#!pip install gradio

## Imports

In [2]:
#model training
import tensorflow.keras as keras
from tensorflow.keras.preprocessing import image as im                   #load image
from tensorflow.keras.applications.resnet50 import preprocess_input      #preprocess image


#preprocessing
#-------------------------------------------------------------------------
# openCV to concatenate
import cv2

# for image name
import random

# define and move to dataset directory
import os

# the directory where the data is stored
directory = "C:/Users/user/Desktop/Dataset"

#for stopping code
import sys
#-------------------------------------------------------------------------

## Proposed model

In [3]:
def create_model():
    resnet50 = keras.applications.resnet50
    
    conv_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

    for layer in conv_model.layers:
        layer.trainable = False

    x = keras.layers.GlobalAveragePooling2D()(conv_model.output)
    x = keras.layers.Dense(512, activation='relu')(x)
    x = keras.layers.Dense(256, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.2)(x)
    predictions = keras.layers.Dense(3, activation='softmax')(x)

    full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)

    return full_model

## Apply preprocessing to 4 inputs

In [4]:
def purepurocessu(img, img2, img3, img4):
    
    #cropping
    #-------------------------------------------------------------------------------------------------------
    #crop width and height
    width = 3600
    height = 3600
    
    im1 = img
        
    center = (im1.shape[0] / 2, im1.shape[1] / 2)
    #print(center)
    x = center[0] - width/2
    y = center[1] - height/2
        
    #slicing to perform cropping
    crop_img = im1[int(y):int(y+width), int(x):int(x+height)]
    
    im2 = img2
        
    center = (im2.shape[0] / 2, im2.shape[1] / 2)
    #print(center)
    x = center[0] - width/2
    y = center[1] - height/2
        
    #slicing to perform cropping
    crop_img2 = im2[int(y):int(y+width), int(x):int(x+height)]
    
    im3 = img3
        
    center = (im3.shape[0] / 2, im3.shape[1] / 2)
    #print(center)
    x = center[0] - width/2
    y = center[1] - height/2
        
    #slicing to perform cropping
    crop_img3 = im3[int(y):int(y+width), int(x):int(x+height)]
    
    im4 = img4
        
    center = (im4.shape[0] / 2, im4.shape[1] / 2)
    #print(center)
    x = center[0] - width/2
    y = center[1] - height/2
        
    #slicing to perform cropping
    crop_img4 = im4[int(y):int(y+width), int(x):int(x+height)]
    #-------------------------------------------------------------------------------------------------------
    
    #concatenate and resize
    #-------------------------------------------------------------------------------------------------------
    #resize for ResNet50
    width = 224
    height = 224

    dim = (width,height)
            
    im_v = cv2.vconcat([cv2.resize(im1, dim), cv2.resize(im2, dim)])
    im_v2 = cv2.vconcat([cv2.resize(im3, dim), cv2.resize(im4, dim)])
    im_v3 = cv2.hconcat([im_v, im_v2])
    
    im_v3 = cv2.resize(im_v3, dim)
    
    #-------------------------------------------------------------------------------------------------------
    
    return im_v3   

## UI using Gradio Library

In [5]:
import gradio as gr
import numpy as np

labels = ['Balut/Penoy','Salted egg','Table egg']

def duckeggpred(angle_1,angle_2,angle_3,angle_4):
    
    full_model = None
    predictionu = None
    
    img = purepurocessu(angle_1,angle_2,angle_3,angle_4)
    
    img = img.reshape((-1, 224, 224, 3))   
    
    full_model = create_model()
    
    save_dir = "C:/Users/user/Desktop/Model/saved_finalmodel_1"
    
    # Load the best model instance to evaluate the performance of the model
    full_model.load_weights(save_dir+"/"+ "finalmodel_1.h5")
    
    predictionu = full_model.predict(img, verbose=0).flatten() 
    #print(predictionu)
    
    return {labels[i]: float(predictionu[i]) for i in range(len(labels))}

iface = gr.Interface(duckeggpred, 
                     [gr.inputs.Image(),gr.inputs.Image(),gr.inputs.Image(),gr.inputs.Image()], 
                     gr.outputs.Label(num_top_classes = 3),
                     title="""
                           Duck Egg Classification Based on its Shell Visual Property through Transfer Learning Using 
                           ResNet50
                           """,
                     description="""
                                 This is a prototype that will predict the duck egg images (2 angles) to 3 classes 
                                 (Balut/Penoy, Salted egg, Table egg). It will use the weights of the results of 
                                 training our model using our dataset in order to do the prediction.
                                 """
                     ,)
iface.launch()

Running locally at: http://127.0.0.1:7860/
To create a public link, set `share=True` in `launch()`.
Interface loading below...


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7860/', None)